# Python Subprocesses

## 1. Running System Commands in Python

Welcome back. We hope you're feeling good about streams, arguments, and exits. Now, we're going to submerge ourselves in subprocesses.

Up to now, we've been using Python to interact with the operating system through baked in functionality. For example, we've used file objects to read the contents of files. Use the `shutil` tool module to check if the disk is full. And use a `sys` module to process standard input, get parameters, or generate an exit code. But what if we needed to run a system program from a Python script? 

Say, for example, that as part of a Python script, we needed to send **ICMP** packets to a host to check if it's responding. We could try to look for an external module that provides this functionality. Or we can just run the `ping` command, which will send packets for us. 

Sometimes it's easier or faster to use a system command as part of our Python script to accomplish a task, or use some functionality that doesn't exist in the Python modules, neither built in or external. For these cases, Python provides a way to execute system commands in our scripts, using functions provided by the `subprocess` module. Let's check out an example. First, we'll import a subprocess module, and then we'll call the date command, which shows the current date using the `subprocess.run` function.

```python
>>> import subprocess
>>> subprocess.run(['date'])
```

```
Tue 07 Jan 2020 03:21:43 PM PST
CompletedProcess(args=['date'], returncode=0)
```

The run function returns an object of the CompletedProcess type. This object includes information related to the execution of the command. From the information that got printed we can see that the returncode of the command was 0. To run the external command a secondary environment is created for the child process or subprocess where the command is executed. While the parent process, which is our script, is waiting on the subprocess to finish, it's blocked, which means that the parent can't do any work until the child finishes. And I bet a lot of parents out there saying, I know that's right. After the external command completes its work, the child process exits and the flow of control returns to the parent. Then the script can continue with normal execution. Let's see this in action by calling the sleep command, which waits for a number of seconds that we tell it before returning.

```python
subprocess.run(['sleep', '2'])
```
```
CompletedProcess(args=['sleep', '2'], returncode=0)
```

You may have noticed that while the sleep command was running, the interpreter was blocked and we couldn't interact with it. That's exactly what we mean about the parent process being blocked until the child process is done.

Check out how we call the command. The run function receives a list that starts with the name of the command that we want to call, followed by any other parameters that we want to pass to that command. So any elements following the program name are the command-line arguments for it. In this case, we're requesting sleep to wait for two seconds. In the last two examples, the commands executed successfully, and so the returncode inside the completed process instance was 0. Let's check out an example where the exit status isn't 0.

If we call LS with a file name that doesn't exist, LS will print an error and return an exit status different than 0. This will be stored in the return code attribute of the completed process instance, and we can access that value in our code

```python
result = subprocess.run(['ls', 'this_nonexistent_file'])
```
```
ls: cannot access 'this_nonexistent_file': No such file or directory
2
```

We can see that the command failed and the returncode stored was 2, letting us know that there was an error. We could use this information in the script to do something different in case the failure. 

Using the `run` function like this is useful if we just want to run a command and only care about whether or not it was successful. The output of the command will be printed to the screen, which means that our script has no control over it. This can be handy for system commands that either don't have useful output like `cp, chmod, sleep,` and many others, or when we don't care about processing the output any further. In other words, when it's just fine to have the output, print it to the screen. For example, if we're writing a script that's changing the permissions of a bunch of files in a tree of directories, we don't care about the output of the chmod command. We only want to know if it was successful or not. If instead, we want to capture the output of an external command and then operate with the results, we need a different strategy.

## 2. Obtaining the Output of a System Command

If we want our Python scripts to manipulate the output of system command that we're executing, we need to tell the run function to capture it for us. This might be helpful when we need to extract information from a command and then use it for something else in our script. For example, say you wanted to create some stats on which users are logging into a server throughout the day. 

You could do this with a script that calls the `who` command, which prints the users currently logged into a computer. The script could parse the output of the command, storing the list of logged-in users once per hour and at the end of the date to generate a daily report. To be able to process the output of commands, we'll set a parameter called `capture_output` to true when calling the run function. 

For our next example, we'll call the host command, which can convert a `host` name to an IP address and vice versa. When calling it, we'll pass the capture output=true parameter and store the result in a variable so that we can access it. Let's give it a try.

```python
>>> import subprocess
>>> result = subprocess.run(['host', '8.8.8.8'], capture_output=True)
```

We know that the result variable is a completed process instance that we can access. We can check the return code attribute like we did before.

```python
>>> print(result.returncode)
0
```

We can also print and operate with the output generated by the command, which is stored in the std out attribute

```python
>>> print(result.stdout)
b'8.8.8.8.in-addr.arpa domain name pointer dns.google.\n'
```

What's that B at the beginning of the string? Well, that B tells us that this string is not a proper string for Python. It's actually an array of bytes. An array of bytes, this is a complex subject so listen closely. 

Data in computers is stored and transmitted in bytes and each can represent up to 256 characters. But there are thousands of possible characters out there used to write in various languages. Chinese, for example, requires over 10,000 different characters. To be able to write in those languages, several specifications called encodings have been created over time to indicate which sequences of bytes represent which characters. 

Nowadays, most people use UTF-8 encoding, which is part of the Unicode standard that lists all the possible characters that can be represented. So going back to our example when we execute the command using run, Python doesn't know which encoding to use to process the output of the command. So it simply represents it as a series of bytes. If we want this to become a proper string, we can call the `decode` method. This method applies an encoding to transform the bytes into a string. By default, it uses a UTF-8 encoding which is what we want. So with all that said, let's transform our array of bytes into a string and then split it into several pieces.

```python
>>> print(result.stdout.decode().spliy)
['8.8.8.8.in-addr.arpa', 'domain', 'name', 'pointer', 'dns.google.']
```

In this way, we're operating with the output of the command that we ran, and we can do whatever we need to do with it. For example, we can choose to keep the last element of the list, which is the name that corresponds to the IP that we're looking for. 

Great. So we just looked at the captured standard output. But what about standard error? If we use the capture output parameter and the command writes any output to standard error, it will be stored in the `stderr` attribute of the completed process instance. Let's look at an example of this. We'll try executing the `rm` command, which we use for removing files passing a filename that doesn't exist.

```python
>>> result = subprocess.run(['rm', 'does_not_exist'], capture_output=True)
>>> print(result.returncode)
1
```

Okay. It failed. Just like we expected. Now, let's check the contents of the stdout and stderr attributes.

```
>>> print(result.stdout)
b''
>>> print(result.stderr)
b"rm: cannot remove 'does_not_exist': No such file or directory\n"
```

So we see that in this case, the standard output was empty. But there was an error printed, a standard error which we can access through the stderr attribute. This is a great example of how standard output and standard error are actually different streams. So Python captures them separately. Okay. We've now seen that we can execute system commands from Python and check whether they succeeded or failed. We've also seen how to capture the standard output and standard error streams so we can use their content in our scripts. These skills can be super useful when writing scripts that your system commands for some involved task and letting our Python scripts cover a broader range of tasks. In the next video, we'll wrap up our discussion on sub-process module by learning some more advanced things that we can do when calling external commands.